In [44]:
import pandas as pd


data = pd.read_csv('/content/train.csv')
print(data)

#csv 데이터 전처리

print(data.isnull().sum())

#평균 구해서 빈칸 채우기 (or 빈칸 데이터가 적은경우는 삭제해도됨.)
avg = data['Age'].mean()  #평균
print(avg)

# 최빈값
최빈값 = data['Embarked'].mode()
print(최빈값)

data['Age'].fillna(value=30, inplace=True)
data['Embarked'].fillna(value='S', inplace=True)  # 최대빈도수인 S로 빈칸 채우기



     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [45]:
import tensorflow as tf
from sklearn.model_selection import train_test_split


# tf.data.Dataset.from_tensor_slices(x데이터, 정답)
# 여기서 x데이터는 csv 데이터에서 정답(정답 column)을 제외한 나머지 데이터들.

정답 = data.pop('Survived')
#ds = tf.data.Dataset.from_tensor_slices( (dict(data)), 정답)
print(data)
ds = tf.data.Dataset.from_tensor_slices( (dict(data),정답) )
print(ds)


     PassengerId  Pclass                                               Name  \
0              1       3                            Braund, Mr. Owen Harris   
1              2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2              3       3                             Heikkinen, Miss. Laina   
3              4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4              5       3                           Allen, Mr. William Henry   
..           ...     ...                                                ...   
886          887       2                              Montvila, Rev. Juozas   
887          888       1                       Graham, Miss. Margaret Edith   
888          889       3           Johnston, Miss. Catherine Helen "Carrie"   
889          890       1                              Behr, Mr. Karl Howell   
890          891       3                                Dooley, Mr. Patrick   

        Sex   Age  SibSp  Parch            Ticket  

In [ ]:
# feature column
feature_columns = []

# tf.keras.layers.DenseFeatures(feature_columns)

# 숫자로 집어넣기 Fare, Parch, SibSp   numeric_column

def nor_fare(x):
  # 노멀라이저 함수
  최대 = data['Fare'].max()
  최소 = data['Fare'].min()  #최소값
  return (x - 최소) / (최대 - 최소)

feature_columns.append(tf.feature_column.numeric_column('Fare',normalizer_fn=nor_fare))  #숫자로 (정수나 실수)
feature_columns.append(tf.feature_column.numeric_column('Parch'))  #
feature_columns.append(tf.feature_column.numeric_column('SibSp'))  #


# 뭉퉁그려서 집어넣기 Age   : bucketized_column
Age = tf.feature_column.numeric_column('Age')
Age_bucket = tf.feature_column.bucketized_column(Age,boundaries=[10,20,30,40,50,60])
feature_columns.append(Age_bucket)

print(Age)
print(feature_columns)


In [ ]:
# 종류 몇개 없어서 카테고리화 집어넣기 (원핫인코딩)  Sex Embarked Pclass(좌석 성급)   : indicator_column
vocab = data['Sex'].unique()
cat = tf.feature_column.categorical_column_with_vocabulary_list('Sex', vocab)
one_hot = tf.feature_column.indicator_column(cat)
feature_columns.append(one_hot)
#print(vocab)

vocab = data['Embarked'].unique()
cat = tf.feature_column.categorical_column_with_vocabulary_list('Embarked', vocab)
one_hot = tf.feature_column.indicator_column(cat)
feature_columns.append(one_hot)

vocab = data['Pclass'].unique()
cat = tf.feature_column.categorical_column_with_vocabulary_list('Pclass', vocab)
one_hot = tf.feature_column.indicator_column(cat)
feature_columns.append(one_hot)



# 종류가 많은 카테고리 (종류가 많아 원핫인코딩하면 비효율적) ex) 티켓종류   Ticket   : embedding_column  원핫인코딩인 아닌 하나의 행렬로 카테고리화
vocab = data['Ticket'].unique()
cat = tf.feature_column.categorical_column_with_vocabulary_list('Ticket', vocab)
one_hot = tf.feature_column.embedding_column(cat, dimension=9)  # dimension 은 크기 얼마의 행렬을 만들건지
feature_columns.append(one_hot)


In [ ]:
# feature column의 모양 확인하기.
ds_batch = ds.batch(32)
feature_layer = tf.keras.layers.DenseFeatures(tf.feature_column.numeric_column('Age'))
feature_layer(next(iter(ds_batch))[0])

In [48]:
# 모델 만들기 / 학습

model = tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(feature_columns),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['acc'])

ds_batch = ds.batch(32)  # 텐서플로우 dataset에 batch 뽑고싶은 경우

# validation data 넣기
# batch로 쪼갠 데이터셋은 validation_split 불가능 , numpy array나 tensor 자료형만 가능

model.fit(ds_batch ,shuffle=True, epochs = 20)


Epoch 1/20


28/28 [==============================] - 1s 3ms/step - loss: 0.6687 - acc: 0.6139
Epoch 2/20
28/28 [==============================] - 0s 3ms/step - loss: 0.6438 - acc: 0.6420
Epoch 3/20
28/28 [==============================] - 0s 3ms/step - loss: 0.6296 - acc: 0.6521
Epoch 4/20
28/28 [==============================] - 0s 3ms/step - loss: 0.6228 - acc: 0.6667
Epoch 5/20
28/28 [==============================] - 0s 3ms/step - loss: 0.6110 - acc: 0.6678
Epoch 6/20
28/28 [==============================] - 0s 3ms/step - loss: 0.6035 - acc: 0.6768
Epoch 7/20
28/28 [==============================] - 0s 3ms/step - loss: 0.5966 - acc: 0.6947
Epoch 8/20
28/28 [==============================] - 0s 3ms/step - loss: 0.5862 - acc: 0.6981
Epoch 9/20
28/28 [==============================] - 0s 3ms/step - loss: 0.5854 - acc: 0.7082
Epoch 10/20
28/28 [==============================] - 0s 3ms/step - loss: 0.5829 - acc: 0.7104
Epoch 11/20
28/28 [==============================] - 0s 3ms/step - loss: 0.5763 